# 1. Đồ án Thu thập và tiền xử lý dữ liệu
tên đề tài: UNSW-NB15 for Network Intrusion Detection Systems

Nhóm 28

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set()

**NOTE: This jupyter notebook was run in a Kaggle environment because it provides better resources (CPU, memory and GPU) to work with datasets that have millions of records**

## Data Validation and Cleaning

In [ ]:
col_names = ['srcip','sport','dstip','dsport','proto','state','dur','sbytes','dbytes','sttl','dttl','sloss','dloss','service','sload','dload','spkts','dpkts',
            'swin','dwin','stcpb','dtcpb','smeansz','dmeansz','trans_depth','res_bdy_len','sjit','djit','stime','ltime','sintpkt','dintpkt','tcprtt','synack',
            'ackdat','is_sm_ips_ports','ct_state_ttl','ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd','ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ltm',
            'ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','attack_cat','label']

useCols = ['proto','state','sbytes','dbytes','sttl','dttl','sloss','dloss','service','sload','dload','spkts','dpkts',
            'swin','dwin','stcpb','dtcpb','smeansz','dmeansz','trans_depth','res_bdy_len','sjit','djit','sintpkt','dintpkt','tcprtt','synack',
            'ackdat','is_sm_ips_ports','ct_state_ttl','ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd','ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ltm',
            'ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','attack_cat','label']

**Name:    Description<br>**
srcip Source IP address<br>
sport Source port number<br>
dstip Destination IP address<br>
dsport Destination port number<br>
proto:   Transaction protocol<br>
state:   Indicates to the state and its dependent protocol, e.g. ACC, CLO, CON, ECO, ECR, FIN, INT, MAS, PAR, REQ, RST, TST, TXD, URH, URN, and (-) (if not used state)<br>
dur:     Record total duration<br>
sbytes:  Source to destination transaction bytes<br> 
dbytes:  Destination to source transaction bytes<br>
sttl:    Source to destination time to live value<br> 
dttl:    Destination to source time to live value<br>
sloss:   Source packets retransmitted or dropped<br>
dloss:   Destination packets retransmitted or dropped<br>
service: http, ftp, smtp, ssh, dns, ftp-data ,irc  and (-) if not much used service<br>
sload:   Source bits per second<br>
dload:   Destination bits per second<br>
spkts:   Source to destination packet count<br> 
dpkts:   Destination to source packet count<br>
swin:    Source TCP window advertisement value<br>
dwin:    Destination TCP window advertisement value<br>
stcpb:   Source TCP base sequence number<br>
dtcpb:   Destination TCP base sequence number<br>
smeansz: Mean of the packet size transmitted by the source<br> 
dmeansz: Mean of the packet size transmitted by the destination<br> 
trans_depth: Represents the pipelined depth into the connection of http request/response transaction<br>
res_bdy_len: Actual uncompressed content size of the data transferred from the server’s http service<br>
sjit: Source jitter (mSec)<br>
djit: Destination jitter (mSec)<br>
Stime: record start time<br>
Ltime: record last time<br>
sintpkt: Source interpacket arrival time (mSec)<br>
dintpkt: Destination interpacket arrival time (mSec)<br>
tcprtt: TCP connection setup round-trip time, the sum of ’synack’ and ’ackdat’<br>
synack: TCP connection setup time, the time between the SYN and the SYN_ACK packets<br>
ackdat: TCP connection setup time, the time between the SYN_ACK and the ACK packets<br>
is_sm_ips_ports: If source (1) and destination (3)IP addresses equal and port numbers (2)(4)  equal then, this variable takes value 1 else 0<br>
ct_state_ttl: No. for each state (6) according to specific range of values for source/destination time to live (10) (11)<br>
ct_flw_http_mthd: No. of flows that has methods such as Get and Post in http service<br>
is_ftp_login: If the ftp session is accessed by user and password then 1 else 0<br>
ct_ftp_cmd: No of flows that has a command in ftp session<br>
ct_srv_src: No. of connections that contain the same service (14) and source address (1) in 100 connections according to the last time (26)<br>
ct_srv_dst: No. of connections that contain the same service (14) and destination address (3) in 100 connections according to the last time (26)<br>
ct_dst_ltm: No. of connections of the same destination address (3) in 100 connections according to the last time (26)<br>
ct_src_ltm: No. of connections of the same source address (1) in 100 connections according to the last time (26)<br>
ct_src_dport_ltm: No of connections of the same source address (1) and the destination port (4) in 100 connections according to the last time (26)<br>
ct_dst_sport_ltm: No of connections of the same destination address (3) and the source port (2) in 100 connections according to the last time (26)<br>
ct_dst_src_ltm: No of connections of the same source (1) and the destination (3) address in in 100 connections according to the last time (26)<br>
attack_cat: The name of each attack category. In this data set , nine categories e.g. Fuzzers, Analysis, Backdoors, DoS Exploits, Generic, Reconnaissance, Shellcode and Worms<br>
Label: 0 for normal and 1 for attack records<br>

**We will exclude the fields 'srcip','sport','dstip','dsport' to make the analysis independent of the source and destination IP, and also skip the fields 'stime','ltime', and 'dur' which are the timestamp of start, end, and duration of flow capture**

In [ ]:
# The column ct_ftp_cmd has some invalid inputs, we will convert the numeric features and overwrite with null values the rest
def castNumeric(item):
    if item.isnumeric():
        return int(item)
    return np.nan

#### Loading CSV Files
**For each CSV file we need to check the services, http along with the column ct_flw_http_mthd because there needs to be a match between the http flow and http service. In the same manner, we need to match the FTP service with the column is_ftp_login. Additionally, the column is_tfp_login should only contain values 0 or 1**

**CSV File 1**

In [ ]:
df1 = pd.read_csv('../input/unsw-nb15/UNSW-NB15_1.csv',names=col_names,usecols=useCols,dtype={'attack_cat':str,'is_sm_ips_ports':int,'is_ftp_login:':int})
df1.info()

In [ ]:
df1[df1['service']=='http']

In [ ]:
df1[df1['service']=='http']['ct_flw_http_mthd'].isna().sum()

In [ ]:
df1[df1['service']=='ftp']

In [ ]:
df1[df1['service']=='ftp']['is_ftp_login'].value_counts()

In [ ]:
df1[df1['service']=='ftp']['is_ftp_login'].isna().sum()

In [ ]:
df1[df1['service']=='ftp']['ct_ftp_cmd'].isna().sum()

**is_sm_ips_ports should only contain values between 0 and 1**

In [ ]:
df1['is_sm_ips_ports'].value_counts()

**The column attack_cat has null values that correspond to normal flows**

In [ ]:
df1['attack_cat'].isna().sum()

In [ ]:
df1['label'].value_counts()

**We also noticed that some categories have an extra space, we will fix that with strip()**

In [ ]:
df1['attack_cat'] = df1['attack_cat'].str.strip()

In [ ]:
df1['attack_cat'].value_counts()

**We will replace null values in column 'attack_cat' with 'normal' category**

In [ ]:
df1['attack_cat'].replace(np.nan,'normal',inplace=True)

In [ ]:
df1['attack_cat'].value_counts()

In [ ]:
df1.info()

**CSV File 2**

In [ ]:
df2 = pd.read_csv('../input/unsw-nb15/UNSW-NB15_2.csv',names=col_names,usecols=useCols,dtype={'attack_cat':str,'is_sm_ips_ports':int,'is_ftp_login:':int},converters={'ct_ftp_cmd':castNumeric})
df2.info()

In [ ]:
df2[df2['service']=='http']

In [ ]:
df2[df2['service']=='http']['ct_flw_http_mthd'].isna().sum()

**There are no missing values for the column 'ct_flw_http_mthd' when the service is http, so we can fill all the NaN values with 0**

In [ ]:
df2['ct_flw_http_mthd'].replace(np.nan,0,inplace=True)

**We need to check if the column 'is_ftp_login' is only 0 or 1 for the service ftp and drop the null values**

In [ ]:
df2[df2['service']=='ftp']

In [ ]:
df2[df2['service']=='ftp']['is_ftp_login'].isna().sum()

In [ ]:
df2[df2['service']=='ftp']['ct_ftp_cmd'].isna().sum()

In [ ]:
df2[df2['service']=='ftp'].dropna(subset=['is_ftp_login','ct_ftp_cmd'],inplace=True)

In [ ]:
df2[df2['service']=='ftp']['is_ftp_login'].value_counts()

**Filtering values where is_ftp_login is not 0 or 1**

In [ ]:
df2 = df2[~((df2['service']=='ftp')&(df2['is_ftp_login']>1))]

In [ ]:
df2[df2['service']=='ftp']['is_ftp_login'].value_counts()

In [ ]:
df2[['is_ftp_login','ct_ftp_cmd']] = df2[['is_ftp_login','ct_ftp_cmd']].fillna(0)

**is_sm_ips_ports should only contain values between 0 and 1**

In [ ]:
df2['is_sm_ips_ports'].value_counts()

**Verifying the column attack_cat has null values that correspond to normal flows**

In [ ]:
df2['attack_cat'].isna().sum()

In [ ]:
df2['label'].value_counts()

In [ ]:
#We also noticed that some categories have an extra space, we will fix that with strip()
df2['attack_cat'] = df2['attack_cat'].str.strip()   
df2['attack_cat'].value_counts()

In [ ]:
#We will replace null values in column 'attack_cat' with 'normal' category
df2['attack_cat'].replace(np.nan,'normal',inplace=True)
df2['attack_cat'].value_counts()
 

In [ ]:
df2.info()

**CSV File 3**

In [ ]:
df3 = pd.read_csv('../input/unsw-nb15/UNSW-NB15_3.csv',names=col_names,usecols=useCols,dtype={'attack_cat':str,'is_sm_ips_ports':int,'is_ftp_login:':int},converters={'ct_ftp_cmd':castNumeric})
df3.info()

In [ ]:
df3[df3['service']=='http']

In [ ]:
df3[df3['service']=='http']['ct_flw_http_mthd'].isna().sum()

**Dropping null values in column ct_flw_http_mthd that belongs to http**

In [ ]:
df3 = df3[~((df3['service']=='http')&(df3['ct_flw_http_mthd'].isna()==True))]

In [ ]:
df3['ct_flw_http_mthd'].replace(np.nan,0,inplace=True)

**We need to check if the column 'is_ftp_login' is only 0 or 1 for the service ftp and drop the null values**


In [ ]:
df3[df3['service']=='ftp']

In [ ]:
df3[df3['service']=='ftp']['is_ftp_login'].isna().sum()

In [ ]:
df3[df3['service']=='ftp']['ct_ftp_cmd'].isna().sum()

In [ ]:
df3[df3['service']=='ftp'].dropna(subset=['is_ftp_login','ct_ftp_cmd'],inplace=True)

In [ ]:
df3[df3['service']=='ftp']['is_ftp_login'].value_counts()

In [ ]:
df3 = df3[~((df3['service']=='ftp')&(df3['is_ftp_login']>1))]

In [ ]:
df3[df3['service']=='ftp']['is_ftp_login'].value_counts()

In [ ]:
df3[['is_ftp_login','ct_ftp_cmd']] = df3[['is_ftp_login','ct_ftp_cmd']].fillna(0)

**is_sm_ips_ports should only contain values between 0 and 1**

In [ ]:
df3['is_sm_ips_ports'].value_counts()

**The column attack_cat has null values that correspond to normal flows**

In [ ]:
df3['attack_cat'].isna().sum()
df3['label'].value_counts()

In [ ]:
#We also noticed that some categories have an extra space, we will fix that with strip()
df3['attack_cat'] = df3['attack_cat'].str.strip()   
df3['attack_cat'].value_counts()

In [ ]:
#We will replace null values in column 'attack_cat' with 'normal' category
df3['attack_cat'].replace(np.nan,'normal',inplace=True)
df3['attack_cat'].value_counts()

In [ ]:
df3.info()

**CSV File 4**

In [ ]:
df4 = pd.read_csv('../input/unsw-nb15/UNSW-NB15_4.csv',names=col_names,usecols=useCols,dtype={'attack_cat':str,'is_sm_ips_ports':int,'is_ftp_login:':int},converters={'ct_ftp_cmd':castNumeric})
df4.info()

In [ ]:
df4[df4['service']=='http']

In [ ]:
df4[df4['service']=='http']['ct_flw_http_mthd'].isna().sum()

In [ ]:
df4['ct_flw_http_mthd'].replace(np.nan,0,inplace=True)

**We need to check if the column 'is_ftp_login' is only 0 or 1 for the service ftp and drop the null values**

In [ ]:
df4[df4['service']=='ftp']

In [ ]:
df4[df4['service']=='ftp']['is_ftp_login'].isna().sum()

In [ ]:
df4[df4['service']=='ftp']['ct_ftp_cmd'].isna().sum()

In [ ]:
df4[df4['service']=='ftp'].dropna(subset=['is_ftp_login','ct_ftp_cmd'],inplace=True)

In [ ]:
df4[df4['service']=='ftp']['is_ftp_login'].value_counts()

In [ ]:
df4 = df4[~((df4['service']=='ftp')&(df4['is_ftp_login']>1))]

In [ ]:
df4[df4['service']=='ftp']['is_ftp_login'].value_counts()

In [ ]:
df4[['is_ftp_login','ct_ftp_cmd']] = df4[['is_ftp_login','ct_ftp_cmd']].fillna(0)

In [ ]:
#is_sm_ips_ports should only contain values between 0 and 1
df4['is_sm_ips_ports'].value_counts()

In [ ]:
#The column attack_cat has null values that correspond to normal flows
df4['attack_cat'].isna().sum()
df4['label'].value_counts()

In [ ]:
#We also noticed that some categories have an extra space, we will fix that with strip()
df4['attack_cat'] = df4['attack_cat'].str.strip()   
df4['attack_cat'].value_counts()

In [ ]:
#We will replace null values in column 'attack_cat' with 'normal' category
df4['attack_cat'].replace(np.nan,'normal',inplace=True)
df4['attack_cat'].value_counts()

In [ ]:
df4.info()

In [ ]:
# Concatenation of DataFrames
df_attacks = pd.concat([df1,df2,df3,df4])

In [ ]:
df_attacks = df_attacks.reset_index()

In [ ]:
df_attacks.drop('index',axis=1,inplace=True)

In [ ]:
df_attacks.info()

In [ ]:
df_attacks.isna().sum()

In [ ]:
df_attacks.shape

In [ ]:
# Replacing '-' in state and service for 'other'
df_attacks['state'] = df_attacks['state'].replace('-','other')
df_attacks['service'] = df_attacks['service'].replace('-','other')

## Exploratory Data Analysis

**Grouping by attack category**

In [ ]:
df_attacks[df_attacks['attack_cat']!='normal'].groupby('attack_cat').size()

**We need to group Backdoor and Backdoors into a single category**

In [ ]:
df_attacks['attack_cat'] = df_attacks['attack_cat'].apply(lambda item: 'Backdoor' if item =='Backdoors' else item)

In [ ]:
attack_by_cat = df_attacks[df_attacks['attack_cat']!='normal'].groupby('attack_cat').size().reset_index(name='counts')
attack_by_cat

**Distribution of attacks by category**

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.barplot(x='attack_cat',y='counts',data=attack_by_cat)
ax.set_title('Distribution of attack categories')
ax.set_xlabel('Category')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.show()

**Distribution of source bytes by attack category**

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.kdeplot(x='sbytes',hue='attack_cat',log_scale=True,data=df_attacks[df_attacks['attack_cat']!='normal'])
ax.set_title('Distribution source bytes by attack category')
ax.set_xlabel('Source to destination transaction bytes')
ax.set_ylabel('Density')
plt.xticks(rotation=45)
plt.show()

**Most used services by attack category**

In [ ]:
df_attacks_cat_services = df_attacks[(df_attacks['attack_cat']!='normal')].groupby(['attack_cat','service']).size().reset_index(name='Count')

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(12,12))
ax[0,0] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Analysis'], x='service',y='Count', ax=ax[0,0])
ax[0,0].set_title('Category: Analysis')
ax[0,0].set_xlabel('Service')
ax[0,1] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Backdoor'], x='service',y='Count', ax=ax[0,1])
ax[0,1].set_title('Category: Backdoor')
ax[0,1].set_xlabel('Service')
ax[0,2] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='DoS'], x='service',y='Count', ax=ax[0,2])
ax[0,2].set_title('Category: DoS')
ax[0,2].set_xlabel('Service')
ax[0,2].tick_params(axis='x', labelrotation = 90)
ax[1,0] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Exploits'], x='service',y='Count', ax=ax[1,0])
ax[1,0].set_title('Category: Exploits')
ax[1,0].set_xlabel('Service')
ax[1,0].tick_params(axis='x', labelrotation = 90)
ax[1,1] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Fuzzers'], x='service',y='Count', ax=ax[1,1])
ax[1,1].set_title('Category: Fuzzers')
ax[1,1].set_xlabel('Service')
ax[1,1].tick_params(axis='x', labelrotation = 90)
ax[1,2] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Generic'], x='service',y='Count', ax=ax[1,2])
ax[1,2].set_title('Category: Generic')
ax[1,2].set_xlabel('Service')
ax[1,2].tick_params(axis='x', labelrotation = 90)
ax[2,0] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Reconnaissance'], x='service',y='Count', ax=ax[2,0])
ax[2,0].set_title('Category: Reconnaissance')
ax[2,0].set_xlabel('Service')
ax[2,0].tick_params(axis='x', labelrotation = 90)
ax[2,1] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Shellcode'], x='service',y='Count', ax=ax[2,1])
ax[2,1].set_title('Category: Shellcode')
ax[2,1].set_xlabel('Service')
ax[2,1].tick_params(axis='x', labelrotation = 90)
ax[2,2] = sns.barplot(data=df_attacks_cat_services[df_attacks_cat_services['attack_cat']=='Worms'], x='service',y='Count', ax=ax[2,2])
ax[2,2].set_title('Category: Worms')
ax[2,2].set_xlabel('Service')
ax[2,2].tick_params(axis='x', labelrotation = 90)
plt.tight_layout()

**Top-5 protocols used by attack category**

In [ ]:
df_attacks_cat_proto = df_attacks[(df_attacks['attack_cat']!='normal')].groupby(['attack_cat','proto']).size().reset_index(name='Count')

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(12,12))
ax[0,0] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Analysis'][:5], x='proto',y='Count', ax=ax[0,0])
ax[0,0].set_title('Category: Analysis')
ax[0,0].set_xlabel('Protocol')
ax[0,1] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Backdoor'][:5], x='proto',y='Count', ax=ax[0,1])
ax[0,1].set_title('Category: Backdoor')
ax[0,1].set_xlabel('Protocol')
ax[0,2] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='DoS'][:5], x='proto',y='Count', ax=ax[0,2])
ax[0,2].set_title('Category: DoS')
ax[0,2].set_xlabel('Protocol')
ax[1,0] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Exploits'][:5], x='proto',y='Count', ax=ax[1,0])
ax[1,0].set_title('Category: Exploits')
ax[1,0].set_xlabel('Protocol')
ax[1,1] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Fuzzers'][:5], x='proto',y='Count', ax=ax[1,1])
ax[1,1].set_title('Category: Fuzzers')
ax[1,1].set_xlabel('Protocol')
ax[1,2] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Generic'][:5], x='proto',y='Count', ax=ax[1,2])
ax[1,2].set_title('Category: Generic')
ax[1,2].set_xlabel('Protocol')
ax[2,0] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Reconnaissance'][:5], x='proto',y='Count', ax=ax[2,0])
ax[2,0].set_title('Category: Reconnaissance')
ax[2,0].set_xlabel('Protocol')
ax[2,1] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Shellcode'][:5], x='proto',y='Count', ax=ax[2,1])
ax[2,1].set_title('Category: Shellcode')
ax[2,1].set_xlabel('Protocol')
ax[2,2] = sns.barplot(data=df_attacks_cat_proto[df_attacks_cat_proto['attack_cat']=='Worms'][:5], x='proto',y='Count', ax=ax[2,2])
ax[2,2].set_title('Category: Worms')
ax[2,2].set_xlabel('Protocol')
plt.tight_layout()

**Distribution of states by attack category**

In [ ]:
df_attacks_cat_state = df_attacks[(df_attacks['attack_cat']!='normal')].groupby(['attack_cat','state']).size().reset_index(name='Count')

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(12,12))
ax[0,0] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Analysis'], x='state',y='Count', ax=ax[0,0])
ax[0,0].set_title('Category: Analysis')
ax[0,0].set_xlabel('State')
ax[0,1] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Backdoor'], x='state',y='Count', ax=ax[0,1])
ax[0,1].set_title('Category: Backdoor')
ax[0,1].set_xlabel('State')
ax[0,2] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='DoS'], x='state',y='Count', ax=ax[0,2])
ax[0,2].set_title('Category: DoS')
ax[0,2].set_xlabel('State')
ax[1,0] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Exploits'], x='state',y='Count', ax=ax[1,0])
ax[1,0].set_title('Category: Exploits')
ax[1,0].set_xlabel('State')
ax[1,1] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Fuzzers'], x='state',y='Count', ax=ax[1,1])
ax[1,1].set_title('Category: Fuzzers')
ax[1,1].set_xlabel('State')
ax[1,2] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Generic'], x='state',y='Count', ax=ax[1,2])
ax[1,2].set_title('Category: Generic')
ax[1,2].set_xlabel('State')
ax[2,0] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Reconnaissance'], x='state',y='Count', ax=ax[2,0])
ax[2,0].set_title('Category: Reconnaissance')
ax[2,0].set_xlabel('State')
ax[2,1] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Shellcode'], x='state',y='Count', ax=ax[2,1])
ax[2,1].set_title('Category: Shellcode')
ax[2,1].set_xlabel('State')
ax[2,2] = sns.barplot(data=df_attacks_cat_state[df_attacks_cat_state['attack_cat']=='Worms'], x='state',y='Count', ax=ax[2,2])
ax[2,2].set_title('Category: Worms')
ax[2,2].set_xlabel('State')
plt.tight_layout()

## Summary of EDA:
1. The distribution of attacks types is not uniform, most of them are generic
2. The distributions of source bytes sent by attack type follow a similar trend, there are not noticeable differences between them
3. Most of the attacks use HTTP service (not considering 'other' category)
4. 3pc and argus are protocols commonly used in distributed computing environments. Both are used by the majority of attacking types along with TCP and UDP
5. INT and FIN are the the states in which most connections are.

## Machine Learning model

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

**The model will focus on trying to predict the type of attack or if its normal (target = attack_cat)**

In [ ]:
attack_cat_map = {
    'normal':0,
    'Generic':1,
    'Exploits':2,
    'Fuzzers':3,
    'DoS':4,
    'Reconnaissance':5,     
    'Analysis':6,          
    'Backdoor':7,
    'Shellcode':8,
    'Worms': 9
    }

In [ ]:
df_attacks['attack_cat'] = df_attacks['attack_cat'].replace(attack_cat_map)

In [ ]:
df_attacks.drop('label',axis=1,inplace=True)

In [ ]:
X = df_attacks.drop('attack_cat',axis=1)
y = df_attacks['attack_cat']

In [ ]:
# Train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42)

In [ ]:
X_train_cat = X_train[['service','proto','state']]
X_train_numerical = X_train.drop(['service','proto','state'],axis=1)

**TRAIN SET: Convert categorical values to one-hot encoding**

In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(drop='first'), ['service','proto','state']),
    remainder='passthrough')
X_train_cat_encoded = transformer.fit_transform(X_train_cat)

**TRAIN SET: Scale numerical features**

In [ ]:
scaler = MinMaxScaler()
X_train_numerical_scaled = scaler.fit_transform(X_train_numerical)

In [ ]:
X_train_np = np.hstack((X_train_numerical_scaled,X_train_cat_encoded.toarray()))

In [ ]:
X_train_np.shape

In [ ]:
y_train.shape

**Scaling and one-hot encoding in the test set**

In [ ]:
X_test_cat = X_test[['service','proto','state']]
X_test_numerical = X_test.drop(['service','proto','state'],axis=1)

In [ ]:
X_test_cat_encoded = transformer.transform(X_test_cat)

In [ ]:
X_test_numerical_scaled = scaler.transform(X_test_numerical)

In [ ]:
X_test_np = np.hstack((X_test_numerical_scaled,X_test_cat_encoded.toarray()))

In [ ]:
X_test_np.shape

In [ ]:
y_test.shape

**Dividing Training set in Training and validation sets**

In [ ]:
X_train_np, X_valid_np, y_train, y_valid = train_test_split(X_train_np, y_train, test_size = 0.2, stratify=y_train, random_state = 42)

In [ ]:
print(f"X_train_shape: {X_train_np.shape}, y_train_shape: {y_train.shape}")
print(f"X_valid_shape: {X_valid_np.shape}, y_valid_shape: {y_valid.shape}")
print(f"X_test_shape: {X_test_np.shape}, y_test_shape: {y_test.shape}")

In [ ]:
print(f"X_train_shape: {X_train_np.shape}, y_train_shape: {y_train.shape}")
print(f"X_valid_shape: {X_valid_np.shape}, y_valid_shape: {y_valid.shape}")
print(f"X_test_shape: {X_test_np.shape}, y_test_shape: {y_test.shape}")

**We need to reshape the input before feeding the data into the neural network model**

In [ ]:
X_train_np = X_train_np.reshape(X_train_np.shape[0],1,X_train_np.shape[1])

In [ ]:
X_train_np.shape

In [ ]:
X_valid_np = X_valid_np.reshape(X_valid_np.shape[0],1,X_valid_np.shape[1])

In [ ]:
X_valid_np.shape

**We need to batch and prefetch the training and validation sets. This is because loading millions of rows in memory is sometimes not possible (depends on the computational resources available) and is better to train the model in batches and prefetch each batch to speed up the training**

In [ ]:
y_pred = np.argmax(model.predict(X_test_np),axis=1)

In [ ]:
target_names = [
    'Normal',
    'Generic',
    'Exploits',
    'Fuzzers',
    'DoS',
    'Reconnaissance',   
    'Analysis',          
    'Backdoor',
    'Shellcode',
    'Worms'
]

## Training with rebalancing

In [ ]:
from imblearn.over_sampling import SMOTENC

In [ ]:
df_attacks['attack_cat'].value_counts()

notice normal category has over 2 millions rows, while Worm category(9) has only 162, we will take only a sample of 100K observations of 0 and 1 categories and keep the rest the same

In [ ]:
df_attacks_0 = df_attacks[df_attacks['attack_cat'] == 0].sample(200000)
df_attacks_1 = df_attacks[df_attacks['attack_cat'] == 1].sample(200000)
df_attacks_not_0_1 = df_attacks[(df_attacks['attack_cat'] != 0) & (df_attacks['attack_cat'] != 1)]

In [ ]:
df_attacks_undersampled = pd.concat([df_attacks_0,df_attacks_1,df_attacks_not_0_1])

In [ ]:
df_attacks_undersampled['attack_cat'].value_counts()

Using df_attacks_undersampled has our new dataframe we will proceed to use SMOTE and oversample all the categories except for normal

In [ ]:
X = df_attacks_undersampled.drop('attack_cat',axis=1)
y = df_attacks_undersampled['attack_cat']
# Train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42)

**Rebalancing the training set**

In [ ]:
y_train.value_counts()

In [ ]:
categorical_indices = [0,1,8]
sm = SMOTENC(random_state=42, categorical_features=categorical_indices)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


In [ ]:
X_train_res.info()

In [ ]:
y_train_res.value_counts()

In [ ]:
X_train_res_cat = X_train_res[['service','proto','state']]
X_train_res_numerical = X_train_res.drop(['service','proto','state'],axis=1)

transformer = make_column_transformer(
    (OneHotEncoder(drop='first'), ['service','proto','state']),
    remainder='passthrough')
X_train_res_cat_encoded = transformer.fit_transform(X_train_res_cat)

scaler = MinMaxScaler()
X_train_res_numerical_scaled = scaler.fit_transform(X_train_res_numerical)

X_train_np = np.hstack((X_train_res_numerical_scaled,X_train_res_cat_encoded.toarray()))

In [ ]:
X_test_cat = X_test[['service','proto','state']]
X_test_numerical = X_test.drop(['service','proto','state'],axis=1)

X_test_encoded = transformer.transform(X_test_cat) # need to use fit_transform due to the fact that some categories are not found in the test set

X_test_numerical_scaled = scaler.transform(X_test_numerical)

X_test_np = np.hstack((X_test_numerical_scaled,X_test_encoded.toarray()))

In [ ]:
X_train_np.shape

In [ ]:
X_test_np.shape

In [ ]:
X_train_np, X_valid_np, y_train, y_valid = train_test_split(X_train_np, y_train_res, test_size = 0.2, stratify=y_train_res, random_state = 42)

In [ ]:
X_train_np = X_train_np.reshape(X_train_np.shape[0],1,X_train_np.shape[1])

In [ ]:
X_valid_np = X_valid_np.reshape(X_valid_np.shape[0],1,X_valid_np.shape[1])